# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#weather data file
weather_data_path = "../WeatherPy/weather_data.csv"

#read the data file
weather_data_df = pd.read_csv(weather_data_path, dtype="object", encoding="utf-8")
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ban tak,17.04,99.08,70.21,91.0,94.0,1.63,TH,1603228830.0
1,praia da vitoria,38.73,-27.07,62.6,72.0,40.0,27.49,PT,1603228831.0
2,jamestown,42.1,-79.24,48.0,93.0,90.0,8.05,US,1603228831.0
3,verkhoyansk,67.54,133.38,8.74,90.0,90.0,3.53,RU,1603228831.0
4,saleaula,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps w/ API key
gmaps.configure(api_key=g_key)

#store lat and lng as locations, and Humidity as the weight
locations = weather_data_df[['Lat', 'Lng']].astype(float).dropna()
humidity_w = weather_data_df['Humidity'].astype(float).dropna()

In [4]:
#Create heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_w, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Ideal weather df
ideal_weather_data = weather_data_df.loc[(weather_data_df['Max Temp'] <'80')& 
                                         (weather_data_df['Max Temp'] >'70')&
                                         (weather_data_df['Wind Speed'] < '10')&
                                         (weather_data_df['Cloudiness'] <'50')]
ideal_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
209,victoria,22.29,114.16,73.0,69.0,43.0,1.01,HK,1603228862.0
239,bijawar,24.63,79.5,72.57,32.0,0.0,0.83,IN,1603228901.0
251,tomohon,1.33,124.84,73.4,94.0,40.0,1.12,ID,1603228904.0
502,badou,7.58,0.6,74.93,87.0,100.0,1.5,TG,1603228968.0
538,ende,-8.84,121.66,77.45,73.0,100.0,1.1,ID,1603228977.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create new hotel df
hotel_df = ideal_weather_data
hotel_df.insert(0, "Hotel Name", " ")
hotel_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
209,,victoria,22.29,114.16,73.0,69.0,43.0,1.01,HK,1603228862.0
239,,bijawar,24.63,79.5,72.57,32.0,0.0,0.83,IN,1603228901.0
251,,tomohon,1.33,124.84,73.4,94.0,40.0,1.12,ID,1603228904.0
502,,badou,7.58,0.6,74.93,87.0,100.0,1.5,TG,1603228968.0
538,,ende,-8.84,121.66,77.45,73.0,100.0,1.1,ID,1603228977.0


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

D:\Python\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Python\envs\PythonData\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Missing field/result... skipping.
Missing field/result... skipping.


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [14]:
# Add marker layer ontop of heat map
figure_layout = {
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure

#Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
#Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [10]:
hotel_df


,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
209,M1 Hotel North Point,victoria,22.29,114.16,73.0,69.0,43.0,1.01,HK,1603228862.0
239,,bijawar,24.63,79.5,72.57,32.0,0.0,0.83,IN,1603228901.0
251,Jhoanie Hotel,tomohon,1.33,124.84,73.4,94.0,40.0,1.12,ID,1603228904.0
502,,badou,7.58,0.6,74.93,87.0,100.0,1.5,TG,1603228968.0
538,LCR HOTEL,ende,-8.84,121.66,77.45,73.0,100.0,1.1,ID,1603228977.0
